In [ ]:
# This block is for Colab compatibility
#
# Colab defaults to a Python 2 runtime without a GPU. Make sure 
# your Colab runtime is Python 3, preferably with a GPU. To do 
# this, click Runtime->Change runtime type on the Colab menu. 
# 
# Uncomment these lines if using on Colab to get the files in place
#! git clone https://www.github.com/rmicallef/bfpytorch
#! cp bfpytorch/*.py .
#! pip install -r bfpytorch/requirements.txt

# Federated Machine Learning

This notebook is a demonstration of _federated learning_, an approach to training a machine learning model by combining the results of training local models on multiple devices. This notebook is part of a series intended to demonstrate federated learning, then move on to test some of the bounds of what federated learning can (and can't) do. We'll be using `pytorch` as our machine learning tool.

## The Task: Digit Recognition

For this demonstration, our task is a classic one: the MNIST digit recognition task. Specifically, we need to determine which number is written in an image of a handwritten digit like this one: 

![Handwritten digit "6"](images/example-6.png)

To carry out this task we need a machine learning model to "look" at an image and classify it into a number. We also need and image data to train the model. We're going to use a simple convolutional neural network model from the [`pytorch` example code](https://github.com/pytorch/examples/blob/master/mnist/main.py) (to keep it simple and to keep the focus on federation, we'll be using whatever we can from the standard `pytorch` examples).

## The Approach to Demonstrate: Federation

The federated approach uses several _worker_ devices to train their own local models with independent data. That is, the workers each train their own local model with a subset of the full dataset.

Each worker trains its local model for a bit, then sends its incrementally trained local model back to a _manager_. The manager combines the local models from all of the workers into a common model, then sends the new common model back to the workers. This unit of processing is called a _round_. 

Then another round begins: the workers each train their local copy of the common model (creating another set of individualized local models), send those updated local models back to the manager. Once again the manager combines these individualized local models into a new common model, and pushes the new common model down to the workers. This cycle of rounds repeats as long as is necessary and useful.

# Common Elements

Let's first set a baseline so we have a point of reference for our federated approach. To keep the comparisons useful, we'll use the same overall dataset and neural network in the baseline case as we do in the federated cases. Let's set those up here.

## The Data - Standard MNIST Datasets
Here we get the MNIST data, a training set and a test set, from the `pytorch` package, again using the same transforms and normalization parameters as the ['pytorch' example code](https://github.com/pytorch/examples/blob/master/mnist/main.py). The standard MNIST training and test datasets, contain 60,000 and 10,000 examples, respectively.

Pytorch uses `DataLoader`s for training. We'll be working with a lot of them here. A `DataLoader` contains a `Dataset` and training parameters like `batch_size`. We'll make a couple of standard `DataLoader`s with the parameters used in the [`pytorch` example code](https://github.com/pytorch/examples/blob/master/mnist/main.py). These contain the 60,000-example training `Dataset` and the 10,000-example test `Dataset`.

In [ ]:
from mnist_utils import get_MNIST_dataloaders

default_training_dataloader, default_testing_dataloader = get_MNIST_dataloaders()

default_training_dataset = default_training_dataloader.dataset
default_testing_dataset = default_testing_dataloader.dataset

print('Training dataset size:', len(default_training_dataset))
print('Testing dataset size:', len(default_testing_dataset))

I'm being super explicit about the digit counts and distributions here, because we're going to be doing some fancy footwork with them later on. So we need to know what we're starting with. So let's check out the default MNIST data.

In [ ]:
from mnist_utils import plot_digit_histogram

plot_digit_histogram([default_training_dataloader, default_testing_dataloader], title='MNIST Complete Datasets')

The out-of-the-box MNIST dataset has roughly equal numbers of samples for each digit, i.e., about as many samples of `4`s as `6`s (though the set is a *little* heavy on `1`s and a *little* light on `5`s). The test set (orange) counts are smaller than the training set (blue) counts by a factor of six, which reflects the relative sizes of the datasets.

## Baseline: The Non-Federated Approach
To show baselines of non-federated learning, we'll train a model the traditional way.

### The Perfect World: All the Data in One Place

We're testing an ideal non-federated situation: all of our data is in one place to train a single model. To do this we'll train that single model with all of the data in the MNIST training set, 60,000 examples. We'll set a default number of "rounds" to train. In the non-federated case, a round is equivalent to an epoch. And we do some other pre-training housekeeping.

Our model and optimizer in all of these examples are simpler and faster than the convolutional neural network used in the [`pytorch` sample code](https://github.com/pytorch/examples/blob/master/mnist/main.py). This is just more expedient for demonstration. If you want to tinker with a heavier model, the original sample network is included in a comment in `mnist_utils.py` and the original optimizers are left inline.

In [ ]:
import torch
from mnist_utils import DefaultNet
from mnist_utils import split_dataset

# default training rounds
default_n_rounds = 10

# target for quitting early (helps to compare approaches more quickly)
default_target_accuracy = None

# Get model and test data
model = DefaultNet()
test_samples, test_labels = split_dataset(default_testing_dataset)

# Load model and test data to the GPU if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # smoke 'em if you got 'em
model = model.to(device)
test_samples = test_samples.to(device)
test_labels = test_labels.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.02, momentum=0.5)
#optimizer = torch.optim.Adadelta(model.parameters(), lr=1.) # original sample optimizer
loss_function = torch.nn.CrossEntropyLoss()

print('Device:', device)
print('Model:', model)
print('Optimizer:', optimizer)
print('Loss function:', loss_function)
print("Training dataset size:", len(default_training_dataloader.dataset))
print("Testing dataset size:", len(default_testing_dataset))

Now we train a model without federation. Standard stuff.

In [ ]:
from mnist_utils import print_training_update
from tqdm.notebook import tqdm
from tqdm.notebook import trange

nonfederated_history = {'test_loss': [], 'test_accuracy': []}

for epoch in trange(default_n_rounds):
    
    # train the model
    model.train(True)
    for i, (training_samples, training_labels) in enumerate(default_training_dataloader):
        training_samples = training_samples.to(device)
        training_labels = training_labels.to(device)
        
        optimizer.zero_grad()
        training_predictions = model(training_samples)
        training_loss = loss_function(training_predictions, training_labels)
        training_loss.backward()
        optimizer.step()

    # test the model
    model.train(False)
    with torch.no_grad():
        output = model(test_samples)
        output = output.to(device)
        
        test_loss = loss_function(output, test_labels).item()
        test_predicted_classes = output.argmax(dim=1, keepdim=True)
        n_correct = test_predicted_classes.eq(test_labels.view_as(test_predicted_classes)).sum().item()
    accuracy = (n_correct / len(test_labels)) * 100.

    #record loss and accuracy
    nonfederated_history['test_loss'].append(test_loss)
    nonfederated_history['test_accuracy'].append(accuracy)
    print_training_update('Round {:3}'.format(epoch), nonfederated_history, id(model))
    

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(nonfederated_history['test_loss'], label='Non-Federated')
ax.set_xlabel("Round")
ax.set_ylabel("Loss")
ax.legend();

fig, ax = plt.subplots()
ax.plot(nonfederated_history['test_accuracy'], label='Non-Federated')
ax.set_xlabel("Round")
ax.set_ylabel("Accuracy")
ax.legend();

So there's our baseline ideal-world situation, a non-federated model trained with all 60,000 data points in front of it.

# The Federated Approach

Now that we have a baseline using the traditional, i.e., non-federated, approach, let's see what federation gets us (and doesn't). We'll do the same MNIST task using federation.

## Our Federation Code

The federation code we're using here is a *simulation* of a federated network. It's not *true* federated learning because it uses one machine to simulate several indepedent machines. As a result of being on one machine, it does not have to deal with network communication or the timing problems inherent to real parallelism. But in our toy example the "local" models are independent in that they do not share data nor local models (except to the extent their models are combined) and that is sufficient to demonstrate the principles underlying federated learning.

Our library includes a `FederatedManager` class and a `FederatedWorker` class. A `FederatedManager` contains a common prediction model and creates several independent `FederatedWorker` instances. Each `FederatedWorker` has its own local model and a distinct subset of the MNIST data.

### Comment
This is fine but There's some text I used to explain why this "not true federated learning" thing doesn't mean what you're reading is useless. I like "algorthmically faithful". It's also a bit tricky to explain what this simulation approach _doesn't_ do (network traffic, race conditions, etc.) until you've explained the algorithm. So I think this section needs a bit of work.

Also the heading is not a good indication of its purpose. This is in some ways the most important bit of the article for a reader who is new to FL! It's the bit where we explain what FL is!

Can we use some figures from the report to make it easier to follow?

Also, backing up a bit: I'm not sure you explain _why_ you want to do FL in the first place at any point in this article (privacy, bandwidth, power, etc.) That needs to be somewhere early, probably right at the top, and then repeated immediately after you explain the algorithm: i.e. the algorithm takes care of the concerns I mentioned at the top.

"Our library": unless you move it into a py file it's not really a library.

In each `round` of federated learning, each `FederatedWorker` trains its local model using only the data that that worker recieves, and then sends its trained local model to the `FederatedManager`. The `FederatedManager` combines the models into an updated common model and pushes that new common model down to each `FederatedWorker`. There are a number of algorithms available for combining local models. Our library uses [federated averaging](https://arxiv.org/abs/1602.05629), a very simple element-by-element average of the model's weights.

```
N = sum(u["n_samples"] for u in updates)
        for key, value in self.model.state_dict().items():
            weight_sum = (
                u["state_dict"][key] * u["n_samples"] for u in updates
            )
            value[:] = sum(weight_sum) / N
```

Thats the whole combination function, an item-wise average of the model weights.

## Splitting the Deck Into Piles for Federation

To illustrate federated learning we use several workers, each representing a remote device. We'll use ten workers here, and for now, we'll assume each of our ten workers has roughly equal types and quantities of data to the other nine (to do this we use `p=0` to reflect roughly even distribution of digit samples for any one worker).

We're going to have ten workers. So we allocate each example in the 60,000 sample MNIST training set to one worker. We do this by making ten torch DataLoaders, one for each worker.

In [ ]:
from mnist_utils import make_federated_dataloaders

federated_equal_dataloaders = make_federated_dataloaders(default_training_dataset, p=0.0)

plot_digit_histogram(federated_equal_dataloaders, 'Digit Counts per DataLoader')

Looking at the allocations, we see that each of our ten workers has about the same number of each digit. For example, each worker has about 550 examples of the digit `5`, and about 650 examples of a `1`. These mirror the digit frequencies in the MNIST set.

## Federated Training
Now let's train the federated system.

In [ ]:
from federated import FederatedManager
from federated import FederatedWorker
from federated import plot_managers

federated_equal_manager = FederatedManager('Federated Unbiased', federated_equal_dataloaders, default_testing_dataset, DefaultNet, verbose=True)
federated_equal_manager.learn(default_n_rounds, default_target_accuracy)

# a proxy to compare the nonfederated model to federated managers
nonfederated_manager = FederatedManager('Non-Federated', [default_training_dataloader], default_testing_dataset, DefaultNet)
nonfederated_manager.history = nonfederated_history

In [ ]:
plot_managers([nonfederated_manager, federated_equal_manager])

The federated model converges, though a little slower than the non-federated version. But in the federated model, we don't have to move any data to a central system.

# Federation with Varying Example Distributions Per Worker

Now we've seen federation work with a dataset scattered across a number of workers. And we saw that its performance isn't substantially worse than the non-federated approach, while decentralizing the work of training and limiting the amount of data transfered between the main manager and the workers.

But what if our workers don't have access to equal amounts of data? Let's explore that.

## Spliting the Decks into Uneven Piles

To test this, we need workers to have access to varying numbers of examples across training classes. So let's make a set of dataloaders that will give each worker more examples of a given digit than the other workers.

The parameter `p` sets the degree of bias toward the overweighted digit in each dataloader. Specifically, `p` is the odds of an example from a specific class being selected for a worker from the base dataset. We'll use `0.15` for `p` for now.

This results in a set of stacked dataloaders. The number of dataloaders is equal to the number of classes. Each dataloader is overweighted to exactly one digit class.

In [ ]:
federated_biased_dataloaders = make_federated_dataloaders(default_training_dataset, p=0.15)
worker_example_counts = [len(dataloader.dataset) for dataloader in federated_biased_dataloaders]

print(worker_example_counts)
print(sum(worker_example_counts))

Note that the dataloaders still give each worker roughly the same *total number* of examples. It's the number of examples *per class* that varies. And each of the 60,000 examples in the MNIST training dataset is allocated (without replacement) to a worker, so the total number of examples across workers is 60,000.

Let's look at how the digit examples are allocated within each worker.

In [ ]:
plot_digit_histogram(federated_biased_dataloaders)

We see here that worker 4, shown in purple, has more examples of the digit `4` than other workers. Likewise, worker 6, shown in pink, has more examples of the digit `6` than other workers.

Here are the actual counts from the dataloaders for workers 4 and 6. You can see that the most frequent examples in those dataloaders are `4` and `6`, respectively.

In [ ]:
from mnist_utils import print_dataset_counters

print_dataset_counters(federated_biased_dataloaders[4].dataset)
print_dataset_counters(federated_biased_dataloaders[6].dataset)

And if you're curious, here are all of the worker counts.

In [ ]:
print_dataset_counters([dataloader.dataset for dataloader in federated_biased_dataloaders])

Now let's see how the federated approach performs with variation in the worker datasets.

In [ ]:
federated_biased_manager = FederatedManager('Fed Biased 0.15', federated_biased_dataloaders, default_testing_dataset, DefaultNet)
federated_biased_manager.learn(default_n_rounds, default_target_accuracy)
plot_managers(federated_biased_manager)

The biased federated model converges just fine, but a little slower than the unbiased federated model.

In [ ]:
plot_managers([nonfederated_manager, federated_equal_manager, federated_biased_manager])

Let's take a closer look at how performance changes with increasingly biased data.

In [ ]:
from federated import evaluate_new_manager

bias_comparison_managers = [evaluate_new_manager('Fed Bias '+ str(bias), default_training_dataset, default_testing_dataset, p=bias, n_rounds=default_n_rounds, target_accuracy=default_target_accuracy, model=DefaultNet) for bias in tqdm([0.0, 0.4, 0.8, 0.9, 0.95, 0.98, 0.99, 1.0], desc='Managers')]

plot_managers(bias_comparison_managers)

The federated models converge even with very heavily biased datasets per worker. Unsurprisingly, it takes longer to hit a given target accuracy. But even when the datasets are completely biased, that is, each worker sees examples from *exactly one class*, a federated set of workers can still at least converge.

# Conclusion
We've taken a look at the basics of federated learning using a toy dataset, and explored some of the peculiar effects of data skew on the performance of federation. In the process of writing up this post, we've stumbled onto a number of behaviors of federated learning that we expect to write up in future posts. 